## 第7章: データベース
> artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．  
(略)  
artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

In [1]:
import json
import locale
from pprint import pprint

import redis

### 60. KVSの構築
> Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [2]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)
# redis returns error when you add a value to an existing key and holding the wrong kind of value.
r.flushall()
# python3's `open` default encoding depends on OS.
print(locale.getpreferredencoding())
with open('artist.json', encoding='utf-8') as f:
    for line in f:
        artist = json.loads(line)
        if 'area' in artist:
            # Since I may have same artist name, I use lists.
            r.rpush(artist['name'], artist['area'])
        else:
            r.rpush(artist['name'], 'UNK')

ANSI_X3.4-1968


### 61. KVSの検索
> 60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [3]:
print(*r.lrange('Rock', 0, -1), sep='\n')

b'France'
b'United Kingdom'
b'Germany'
b'United States'
b'United States'


# 62. KVS内の反復処理
> 60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [4]:
# return 1 each area == Japan artist in the list for each key (=artist name).
sum(1 for key in r.keys() for area in r.lrange(key, 0, -1) if area == b'Japan' )

22821

### 63. オブジェクトを値に格納したKVS
> KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [5]:
r = redis.StrictRedis(host='localhost', port=6379, db=1)
r.flushdb()
with open('artist.json', encoding='utf-8') as f:
    for line in f:
        artist = json.loads(line)
        if 'tags' in artist:
            r.hset(artist['name'], 'tags', artist['tags'])
        else:
            r.hset(artist['name'], 'tags', 'UNK')

In [6]:
print(*eval(r.hget('Coldplay', 'tags')), sep='\n')

{'count': 2, 'value': 'rock'}
{'count': 1, 'value': 'pop rock'}
{'count': 1, 'value': 'piano pop'}
{'count': 3, 'value': 'british'}
{'count': 1, 'value': 'parlophone'}
{'count': 1, 'value': 'english'}
{'count': 3, 'value': 'uk'}
{'count': 1, 'value': 'piano rock'}
{'count': 1, 'value': 'britannique'}
{'count': 1, 'value': 'rock and indie'}
{'count': 1, 'value': 'chapel'}
{'count': 2, 'value': 'alternative rock'}
{'count': 1, 'value': 'pop/rock'}


### 64. MongoDBの構築
> アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [7]:
# %%bash
# echo "$(pwd)/data/db" | xargs -I {} mongod -dbpath {} &

In [8]:
from pymongo import MongoClient, DESCENDING

client = MongoClient('localhost', 27017)
mdb = client['artist'] # database
artist = mdb['artist'] # collection
artist.drop()
with open('artist.json', encoding='utf-8') as f:
    for line in f:
        artist_info = json.loads(line)
        artist.insert_one(artist_info)

In [9]:
artist.create_index([('name', DESCENDING)]) 
artist.create_index([('aliases.value', DESCENDING)])
artist.create_index([('tags.value', DESCENDING)])
artist.create_index([('rating.value', DESCENDING)])

'rating.value_-1'

### 65. MongoDBの検索
> MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [10]:
%%bash
mongo
use artist
db.artist.find({'name':'Queen'})

MongoDB shell version v3.6.5
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.5
switched to db artist
{ "_id" : ObjectId("5b3cb1c20454aa0d865e7dff"), "name" : "Queen", "area" : "Japan", "gender" : "Female", "tags" : [ { "count" : 1, "value" : "kamen rider w" }, { "count" : 1, "value" : "related-akb48" } ], "sort_name" : "Queen", "ended" : true, "gid" : "420ca290-76c5-41af-999e-564d7c71f1a7", "type" : "Character", "id" : 701492, "aliases" : [ { "name" : "Queen", "sort_name" : "Queen" } ] }
{ "_id" : ObjectId("5b3cb1c90454aa0d865f44ab"), "rating" : { "count" : 24, "value" : 92 }, "begin" : { "date" : 27, "month" : 6, "year" : 1970 }, "name" : "Queen", "area" : "United Kingdom", "tags" : [ { "count" : 2, "value" : "hard rock" }, { "count" : 1, "value" : "70s" }, { "count" : 1, "value" : "queen family" }, { "count" : 1, "value" : "90s" }, { "count" : 1, "value" : "80s" }, { "count" : 1, "value" : "glam rock" }, { "count" : 4, "value" : "british" }, { "count" : 1, "value

In [11]:
for queen in artist.find({'name':'Queen'}):
    pprint(queen)

{'_id': ObjectId('5b3cb1c20454aa0d865e7dff'),
 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}],
 'area': 'Japan',
 'ended': True,
 'gender': 'Female',
 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
 'id': 701492,
 'name': 'Queen',
 'sort_name': 'Queen',
 'tags': [{'count': 1, 'value': 'kamen rider w'},
          {'count': 1, 'value': 'related-akb48'}],
 'type': 'Character'}
{'_id': ObjectId('5b3cb1c90454aa0d865f44ab'),
 'aliases': [{'name': '女王', 'sort_name': '女王'}],
 'area': 'United Kingdom',
 'begin': {'date': 27, 'month': 6, 'year': 1970},
 'ended': True,
 'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3',
 'id': 192,
 'name': 'Queen',
 'rating': {'count': 24, 'value': 92},
 'sort_name': 'Queen',
 'tags': [{'count': 2, 'value': 'hard rock'},
          {'count': 1, 'value': '70s'},
          {'count': 1, 'value': 'queen family'},
          {'count': 1, 'value': '90s'},
          {'count': 1, 'value': '80s'},
          {'count': 1, 'value': 'glam rock'},
          {'count': 4, 'valu

### 66. 検索件数の取得
> MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

In [12]:
%%bash
mongo
use artist
db.artist.find({'area':'Japan'}).count()

MongoDB shell version v3.6.5
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.5
switched to db artist
22821
bye


### 67. 複数のドキュメントの取得
> 特定の（指定した）別名を持つアーティストを検索せよ．

In [13]:
for artist_info in artist.find({'$or': [{'name': 'ハルイチ'},{'aliases.name': 'ハルイチ'}]}):
    pprint(artist_info)

{'_id': ObjectId('5b3cb1720454aa0d8655dccd'),
 'aliases': [{'name': 'ハルイチ', 'sort_name': 'ハルイチ'},
             {'name': 'Shindo Haruichi', 'sort_name': 'Shindo Haruichi'},
             {'name': '新藤 晴一', 'sort_name': '新藤 晴一'}],
 'begin': {'date': 20, 'month': 9, 'year': 1974},
 'ended': True,
 'gid': '2686d3ca-40ca-4d15-aa55-ea700783cb6f',
 'id': 565379,
 'name': '新藤晴一',
 'sort_name': 'Shindo, Haruichi',
 'type': 'Person'}


### 68. ソート
> "dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [14]:
for artist_info in artist.find({'tags.value': 'dance'}).sort('rating.count', DESCENDING).limit(10):
    print(artist_info['name'])

Madonna
Björk
The Prodigy
Rihanna
Britney Spears
Maroon 5
Adam Lambert
Fatboy Slim
Basement Jaxx
Cornershop


### 69. Webアプリケーションの作成
> ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．

website: https://artist-search-flask.herokuapp.com/  
code: https://github.com/marukaz/artist_search